In [29]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_curve, auc
from sklearn.tree import DecisionTreeClassifier 
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from statsmodels.formula.api import ols
from sklearn.metrics import r2_score


In [20]:
#import all datasets

data = pd.read_csv('Data/spotify_dataframe', index_col = 'Unnamed: 0')
smote_data = pd.read_csv('Data/smote_data', index_col = 'Unnamed: 0')
nm_data = pd.read_csv('Data/nearmiss_data', index_col = 'Unnamed: 0')

data.drop(columns = ['artists', 'popularity', 'genres'], axis = 1, inplace = True)
data.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,count,popularity_labels
0,0.598500,0.470100,267072.000000,0.376203,0.010261,0.283050,-14.434300,0.209150,114.128800,0.358320,5,1,10,Successful
1,0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,5,1,26,Successful
2,0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,0,1,7,Successful
3,0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,0,1,27,Successful
4,0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,5,1,7,Successful


In [11]:
#assign variables
X = data.drop('popularity_labels', axis = 1)
X_smote = smote_data.drop('popularity_labels', axis = 1)
X_nm = nm_data.drop('popularity_labels', axis = 1)

y = data['popularity_labels']
y_smote = smote_data['popularity_labels']
y_nm = nm_data['popularity_labels']

#train, test, split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = .2, random_state = 42)
X_train_smote, X_test_smote, y_train_smote,y_test_smote = train_test_split(X_smote, y_smote, test_size = .2, random_state = 42)
X_train_nm, X_test_nm, y_train_nm, y_test_nm = train_test_split(X_nm, y_nm, test_size = .2, random_state = 42)

In [13]:
#standardize data
#ss = StandardScaler()

#ss_train = ss.fit_transform(X_train)
#ss_train_smote = ss.fit_transform(X_train_smote)
#ss_train_nm = ss.fit_transform(X_train_nm)

### First vanilla models

In [16]:
#realized I could've made my life easier with a pipeline - so going to add that in here! Better late than never
from sklearn.pipeline import Pipeline

pipe = Pipeline([('ss', StandardScaler()),
                 ('tree', DecisionTreeClassifier(random_state=10))])

pipe.fit(X_train, y_train)
pipe.fit(X_train_smote, y_train_smote)
pipe.fit(X_train_nm, y_train_nm)

print('Decision Tree Performance')
print('Original: {}'.format(pipe.score(X_test, y_test)))
print('SMOTE: {}'.format(pipe.score(X_test_smote, y_test_smote)))
print('NearMiss: {}'.format(pipe.score(X_test_nm, y_test_nm)))

Original: 0.7619852489244008
SMOTE: 0.7072391412880679
NearMiss: 0.6145587287953618


In [19]:
pipe_KNN = Pipeline([('ss', StandardScaler()),
                 ('knn', KNeighborsClassifier())])

pipe_KNN.fit(X_train, y_train)
pipe_KNN.fit(X_train_smote, y_train_smote)
pipe_KNN.fit(X_train_nm, y_train_nm)

print('KNN Performance')
print('Original: {}'.format(pipe_KNN.score(X_test, y_test)))
print('SMOTE: {}'.format(pipe_KNN.score(X_test_smote, y_test_smote)))
print('NearMiss: {}'.format(pipe_KNN.score(X_test_nm, y_test_nm)))

Original: 0.6324523663183774
SMOTE: 0.6442336495257114
NearMiss: 0.6055400472407129


### Switch gears to linear regression

In [24]:
data_lin = data = pd.read_csv('Data/spotify_dataframe', index_col = 'Unnamed: 0')

features = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 
           'tempo', 'valence', 'key', 'mode', 'count']

X2 = data_lin[features]
y2 = data_lin['popularity']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = .2, random_state = 42)

In [38]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train2, y_train2)
y_hat = lr.predict(X_test2)

In [39]:
print('R^2: ', r2_score(y_test2, y_hat))

R^2:  0.1813054063079298


In [42]:
formula = 'popularity ~ acousticness +danceability + duration_ms + energy + instrumentalness + liveness + loudness + speechiness + tempo + valence + key + mode + count'

model = ols(formula= formula, data=data_lin).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     544.9
Date:                Sat, 06 Feb 2021   Prob (F-statistic):               0.00
Time:                        18:07:11   Log-Likelihood:            -1.4502e+05
No. Observations:               32539   AIC:                         2.901e+05
Df Residuals:                   32525   BIC:                         2.902e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           53.2806      1.351     39.432      0.000      50.632      55.929
acousticness       -21.6318      0.578    -37.412      0.000     -22.765     -20.498
danceability        -6.2634      0.930     -6.735      0.000      -8.086      -4.441
duration_ms      -7.293e-06   7.58e-07     -9.615      0.000   -8.78e-06   -5.81e-06
energy              -8.1010      1.044     -7.762      0.000     -10.147      -6.055
instrumentalness   -21.9779      0.394    -55.727      0.000     -22.751     -21.205
liveness           -18.2588      0.811    -22.521      0.000     -19.848     -16.670
loudness            -0.2235      0.038     -5.840      0.000      -0.298      -0.148
speechiness        -10.4875      1.127     -9.307      0.000     -12.696      -8.279
tempo               -0.0309      0.005     -6.091      0.000      -0.041      -0.021
valence              3.1239      0.615      5.083      0.000       1.919       4.328
key                 -0.0787      0.034     -2.344      0.019      -0.145      -0.013
mode                 0.7265      0.270      2.692      0.007       0.197       1.256
count                0.0003      0.002      0.146      0.884      -0.004       0.005
==============================================================================
Omnibus:                     1673.772   Durbin-Watson:                   1.872
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1099.053
Skew:                           0.331   Prob(JB):                    2.21e-239
Kurtosis:                       2.390   Cond. No.                     4.41e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.41e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Begin with vanilla models

### K Nearest Neighbors

In [23]:
#start with a KNN classifier

clf_kn= KNeighborsClassifier()

#fit
clf_kn.fit(scaled_train, y_train)

#predict
y_hat = clf_kn.predict(scaled_test)

In [24]:
#check performance

def print_metrics(test, preds):
    print("Precision Score: {}".format(precision_score(test, preds, average = 'macro')))
    print("Recall Score: {}".format(recall_score(test, preds, average = 'macro')))
    print("Accuracy Score: {}".format(accuracy_score(test, preds)))
    print("F1 Score: {}".format(f1_score(test, preds, average = 'macro')))
    
print_metrics(y_test, y_hat)

Precision Score: 0.6165350679293381
Recall Score: 0.6096566549759507
Accuracy Score: 0.6544253226797787
F1 Score: 0.6120948609905715


In [25]:
#consider other k values

def find_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1, 2):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_hat = knn.predict(X_test)
        f1 = f1_score(y_test, y_hat, average = 'macro')
        if f1 > best_score:
            best_k = k
            best_score = f1
    print("Best Value for k: {}".format(best_k))
    print("F1-Score: {}".format(best_score))

In [26]:
find_k(scaled_train, y_train, scaled_test, y_test)

Best Value for k: 13
F1-Score: 0.6240938122700931


In [27]:
#test performance for all metrics with the best k value

clf_bestk = KNeighborsClassifier(n_neighbors = 13)

clf_bestk.fit(scaled_train, y_train)

y_hat_best = clf_bestk.predict(scaled_test)

print_metrics(y_test, y_hat)

Precision Score: 0.6165350679293381
Recall Score: 0.6096566549759507
Accuracy Score: 0.6544253226797787
F1 Score: 0.6120948609905715


##### Thoughts on KNN Outcome:
Not a very good F1, with a slightly better accuracy score. Definitely not great and probably not the model I'll end up using. However, would like to go back and take a look at the cleaning to see what could make this better

Going to make some changes to the dataframe in hopes of improving the KNN model. First thought is to dummy the categorical variables 

In [28]:
dummies = ['key']
df_knn = pd.get_dummies(X, columns = dummies)

In [29]:
df_knn.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0.598500,0.470100,267072.000000,0.376203,0.010261,0.283050,-14.434300,0.209150,114.128800,0.358320,...,0,0,0,1,0,0,0,0,0,0
1,0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,...,0,0,0,1,0,0,0,0,0,0
2,0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,...,0,0,0,0,0,0,0,0,0,0
3,0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,...,0,0,0,0,0,0,0,0,0,0
4,0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,...,0,0,0,1,0,0,0,0,0,0


In [30]:
X_knn = df_knn
y_knn = df['popularity_labels']
len(X_knn)

32539

In [31]:
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y_knn, test_size = .2, random_state = 42)

In [32]:
scaled_train_knn = scaler.fit_transform(X_train_knn)
scaled_test_knn = scaler.fit_transform(X_test_knn)

In [33]:
#using new KNN X data with no adjustment to hyperparameters
clf_knn = KNeighborsClassifier()

clf_knn.fit(scaled_train_knn, y_train)
y_hat_new_knn = clf_knn.predict(scaled_test_knn)

In [34]:
print_metrics(y_test_knn, y_hat_new_knn)

Precision Score: 0.6057185290666891
Recall Score: 0.597671990381845
Accuracy Score: 0.6409035033804549
F1 Score: 0.6001418884190932


In [35]:
#find the best K value and F1 score with the new X data
find_k(scaled_train_knn, y_train_knn, scaled_test_knn, y_test_knn)

Best Value for k: 1
F1-Score: 0.607927114503413


##### Final Thoughts | KNN
KNN certainly doesn't seem very successful for this dataset. This F1 is lower than the vanilla model

### Moving on to Trees!

Will start with the original dataframe with no modification first 
Starting with a tree using entropy considering it's the most common method

In [36]:
clf_tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 10)

clf_tree.fit(X_train, y_train)
y_hat_tree = clf_tree.predict(X_test)

In [37]:
print('Accuracy: {}'.format(accuracy_score(y_test, y_hat_tree)))
print('F1: {}'.format(f1_score(y_test, y_hat_tree, average = 'macro')))

pd.crosstab(y_test, y_hat_tree, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.6439766441303012
F1: 0.6067395009548563


Predicted,Successful,Unsuccessful,Very Successful,All
True,,,,
Successful,850,460,345,1655
Unsuccessful,449,2476,366,3291
Very Successful,324,373,865,1562
All,1623,3309,1576,6508


It's becoming obvious to me that the imbalanced nature of this dataset is affecting my model performance. In the next notebook, I'm going to solve for this problem and rerun my models 

In [5]:
clf = DecisionTreeClassifier(criterion='entropy', random_state = 10)

clf.fit(X_train, y_train)
y_hat_tree = clf.predict(X_test)
y_hat_train = clf.predict(X_train)

In [6]:
print('Accuracy: {}'.format(accuracy_score(y_test, y_hat_tree)))
print('Precision: {}'.format(precision_score(y_test, y_hat_tree, average = 'macro')))
print('Recall: {}'.format(recall_score(y_test, y_hat_tree, average = 'macro')))


pd.crosstab(y_test, y_hat_tree, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.6409547223929523
Precision: 0.5989464108896949
Recall: 0.5989123706150902


Predicted,1,2,3,All
True,,,,
1,3785,705,518,5008
2,688,1205,533,2426
3,531,530,1267,2328
All,5004,2440,2318,9762


In [7]:
#double confirm results
r2 = r2_score(y_test, y_hat_tree)
rmse = mean_squared_error(y_test, y_hat_tree, squared=False)

print('R2 Score: {}'.format(r2))
print('MSE: {}'.format(rmse))

R2 Score: -0.007840745055161635
MSE: 0.8254803100413293


Ok, so with the same cleaning used for logsitic models, we get a very similar precison/ recall output. The Successful class is much cleaner compared to the logistic model - but the overall score still isn't promising. I've never seen a negative R2 before, so need to do research there

In [8]:
pipe = Pipeline([('classifier' , DecisionTreeClassifier())])


param_grid = [
    {'classifier' : [DecisionTreeClassifier()],
     'classifier__criterion' : ['gini', 'entropy'],
    'classifier__class_weight' : ['balanced', 'None'],
    'classifier__min_samples_split' : [1, 2, 4, 8]}]

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)
best_clf = clf.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    9.5s finished


In [9]:
print(best_clf.best_params_)

{'classifier': DecisionTreeClassifier(class_weight='balanced', criterion='entropy'), 'classifier__class_weight': 'balanced', 'classifier__criterion': 'entropy', 'classifier__min_samples_split': 2}


In [10]:
clf = DecisionTreeClassifier(criterion='entropy', class_weight = 'balanced', min_samples_split = 2, random_state = 10)

clf.fit(X_train, y_train)
y_hat_tree = clf.predict(X_test)
y_hat_train = clf.predict(X_train)

print('Accuracy: {}'.format(accuracy_score(y_test, y_hat_tree)))
print('Precision: {}'.format(precision_score(y_test, y_hat_tree, average = 'macro')))
print('Recall: {}'.format(recall_score(y_test, y_hat_tree, average = 'macro')))
print('R2 Score: {}'.format(r2_score(y_test, y_hat_tree)))
print('MSE: {}'.format(mean_squared_error(y_test, y_hat_tree, squared=False)))


pd.crosstab(y_test, y_hat_tree, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.6379840196681008
Precision: 0.597080272607638
Recall: 0.5964182675041684
R2 Score: -0.052687537078061286
MSE: 0.8436464885628417


Predicted,1,2,3,All
True,,,,
1,3762,689,557,5008
2,692,1223,511,2426
3,581,504,1243,2328
All,5035,2416,2311,9762
